In [ ]:
#Dependencies
#
from flask import Flask, render_template, redirect
#connection to MongoDb
from pymongo import MongoClient
# import jupyter notebook as a function
import scrape

In [ ]:
#this creates Flask
Mars_app = Flask(__name__)

# Use pymongo to set up mongo connection
mongo_url = "mongodb://localhost:27017"
#pass Mongo URL to MongoClient constructor. Defines the connection
client = MongoClient(mongo_url)
#defines a unique handle on the Mongo database for this code
db = client.MarsNews_db
#handle to the connection to insert the data. Define the variable
#hemispheres_collection that will contain the data
hemispheres_collection=db.

In [ ]:
def insert_data_Mongo(): 
    facts_lst = [{"name":"key1Hemisphere", "link":""},{"name":"key2Hemisphere", "link":""},
{"name":"key3Hemisphere", "link":""},{"name":"key4Hemisphere", "link":""}]

In [ ]:
hemispheres=[{"name":"key1Hemisphere", "link":""},{"name":"key2Hemisphere", "link":""},
{"name":"key3Hemisphere", "link":""},{"name":"key4Hemisphere", "link":""}]

In [ ]:
#This is the route that will import most of my code
@Mars_app.route("/scrape")
def scrape():
    message = "Welcome to my API"
    return render_template('index.html', text=message, dict=hemispheres)

In [ ]:
@Mars_app.route("/")
def query():
    WhatsNew = db.MarsNews_Mongo.find_one()
    return render_template("index.html", WhatsNew=WhatsNew)


In [ ]:
@Mars_app.route("/bio")
def bio():
    return render_template("bio.html")

In [ ]:
@Mars_app.route("/Sources")
def index():
    Sources_list=["https://mars.nasa.gov/news/",
                  "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars",
                  "https://space-facts.com/mars/"]
    return render_template("sources.html",  Sources=Sources_list)

In [ ]:
if __name__ == "__main__":
    Mars_app.run(debug=True)